In [1]:
#import folder

import pandas as pd
import numpy as np

df = pd.read_csv('../Clean_data/btcfuture_agg.csv')

df

,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume
0,2025-05-01,94125.2,97388.0,94086.7,96423.3,184526.755,2025-05-01 23:59:59.999,1.772449e+10,2825368,96525.420,9.270273e+09
1,2025-05-01,94125.2,97388.0,94086.7,96423.3,184526.755,2025-05-01 23:59:59.999,1.772449e+10,2825368,96525.420,9.270273e+09
2,2025-05-02,96423.2,97865.4,96283.1,96831.2,139197.526,2025-05-02 23:59:59.999,1.350317e+10,2240247,68734.402,6.668888e+09
3,2025-05-02,96423.2,97865.4,96283.1,96831.2,139197.526,2025-05-02 23:59:59.999,1.350317e+10,2240247,68734.402,6.668888e+09
4,2025-05-03,96831.2,96868.4,95700.0,95803.3,71420.629,2025-05-03 23:59:59.999,6.871383e+09,1207124,34360.578,3.306264e+09
...,...,...,...,...,...,...,...,...,...,...,...
149,2025-08-27,111710.0,112568.5,110306.9,111194.9,114205.122,2025-08-27 23:59:59.999,1.273924e+10,2613099,57009.686,6.360193e+09
150,2025-08-28,111194.9,113429.0,110811.0,112507.3,97630.277,2025-08-28 23:59:59.999,1.098862e+10,2252826,48243.477,5.429943e+09
151,2025-08-29,112507.3,112585.7,107411.4,108332.4,179725.172,2025-08-29 23:59:59.999,1.968867e+10,3720115,85606.138,9.377659e+09
152,2025-08-30,108332.4,108879.6,107281.0,108757.9,60656.883,2025-08-30 23:59:59.999,6.572710e+09,1397811,29582.419,3.205458e+09


In [2]:
#Organise list to cal
#adding prep_ to the front of the list for better indication when combine



#Prep Daily Returns with zscore
df["prep_normal_return"] = df["close"].pct_change() * 100   # in %
roll = df["prep_normal_return"].rolling(window=30)
df["Z_prep_30d_return"] = (df["prep_normal_return"] - roll.mean()) / roll.std()


#Perp daily log return
df["prep_log_return"] = np.log(df["close"] / df["close"].shift(1))
roll = df["prep_log_return"].rolling(window=30)
df["Z_prep30d_logreturn"] = (df["prep_log_return"] - roll.mean()) / roll.std()


#Close-Price z-scores 7d/30d
#Rolling Mean & Volatility 
df["prep_7d_rolling_mean"] = df["close"].rolling(window=7).mean()
df["prep_7d_rolling_volatility"] = df["prep_normal_return"].rolling(window=7).std()
df["z_prep_7d"] = (df["close"] - df["close"].rolling(7).mean()) / df["close"].rolling(7).std()


df["prep_30d_rolling_mean"] = df["close"].rolling(window=30).mean()
df["prep_30d_rolling_volatility"] = df["prep_normal_return"].rolling(window=30).std()
df["z_prep_30d"] = (df["close"] - df["close"].rolling(30).mean()) / df["close"].rolling(30).std()



   
#part2 for seperation
#intraday range _ zscore
df["intraday"] = (df["high"] - df["low"]) / df["close"]

df["z_intraday_30"] = (df["intraday"] - df["intraday"].rolling(30).mean()) / df["intraday"].rolling(30).std()

#volume zscore
df["z_prep_volume"] = (df["volume"] - df["volume"].rolling(30).mean()) / df["volume"].rolling(30).std()


#volume %change
df["prep_volume_change_%"] = df["volume"].pct_change() * 100

roll = df["prep_volume_change_%"].rolling(window=30)
df["z_prep_30d_volume_change"] = (df["prep_volume_change_%"] - roll.mean()) / roll.std()

roll = df["prep_volume_change_%"].rolling(window=7)
df["z_prep_7d_volume_change"] = (df["prep_volume_change_%"] - roll.mean()) / roll.std()



#Prep special
#1. Taker-buy ratio (aggression)
df["taker_sell_volume"] = df["volume"] - df["taker_buy_volume"]
df["taker_sell_volume"] = df["volume"] - df["taker_buy_volume"]
df["buy_share"] = (df["taker_buy_volume"] / df["volume"]).clip(0, 1)   # 0..1
df["sell_share"] = 1 - df["buy_share"]
df["OFI"] = 2*df["buy_share"] - 1   # order-flow imbalance, -1..+1



#2. Z-Score for taker buy ratio (30d)


In [3]:
#Prep special
#1. Taker-buy ratio (aggression)
df["taker_sell_volume"] = df["volume"] - df["taker_buy_volume"]
df["taker_sell_volume"] = df["volume"] - df["taker_buy_volume"]
df["buy_share"] = (df["taker_buy_volume"] / df["volume"]).clip(0, 1)   # 0..1
df["sell_share"] = 1 - df["buy_share"]
df["OFI"] = 2*df["buy_share"] - 1   # order-flow imbalance, -1..+1

roll = df["OFI"].rolling(window=30)
df["z_prep_OFI"] = (df["OFI"] - roll.mean()) / roll.std()

#average trade volume
df["prep_average_trade_volume_inU"] = df["quote_volume"] / df["count"]

df["prep_average_trade_volume_inbtc"] = df["volume"] / df["count"]

roll = df["prep_average_trade_volume_inU"].rolling(window=30)
df["z_prep_30d_average_trade_inU"] = (df["prep_average_trade_volume_inU"] - roll.mean()) / roll.std()

df

,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,...,z_prep_30d_volume_change,z_prep_7d_volume_change,taker_sell_volume,buy_share,sell_share,OFI,z_prep_OFI,prep_average_trade_volume_inU,prep_average_trade_volume_inbtc,z_prep_30d_average_trade_inU
0,2025-05-01,94125.2,97388.0,94086.7,96423.3,184526.755,2025-05-01 23:59:59.999,1.772449e+10,2825368,96525.420,...,NaN,NaN,88001.335,0.523097,0.476903,0.046194,NaN,6273.340217,0.065311,NaN
1,2025-05-01,94125.2,97388.0,94086.7,96423.3,184526.755,2025-05-01 23:59:59.999,1.772449e+10,2825368,96525.420,...,NaN,NaN,88001.335,0.523097,0.476903,0.046194,NaN,6273.340217,0.065311,NaN
2,2025-05-02,96423.2,97865.4,96283.1,96831.2,139197.526,2025-05-02 23:59:59.999,1.350317e+10,2240247,68734.402,...,NaN,NaN,70463.124,0.493790,0.506210,-0.012419,NaN,6027.535309,0.062135,NaN
3,2025-05-02,96423.2,97865.4,96283.1,96831.2,139197.526,2025-05-02 23:59:59.999,1.350317e+10,2240247,68734.402,...,NaN,NaN,70463.124,0.493790,0.506210,-0.012419,NaN,6027.535309,0.062135,NaN
4,2025-05-03,96831.2,96868.4,95700.0,95803.3,71420.629,2025-05-03 23:59:59.999,6.871383e+09,1207124,34360.578,...,NaN,NaN,37060.051,0.481102,0.518898,-0.037797,NaN,5692.358776,0.059166,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2025-08-27,111710.0,112568.5,110306.9,111194.9,114205.122,2025-08-27 23:59:59.999,1.273924e+10,2613099,57009.686,...,-0.623971,-0.551059,57195.436,0.499187,0.500813,-0.001626,0.824926,4875.145430,0.043705,-1.028991
150,2025-08-28,111194.9,113429.0,110811.0,112507.3,97630.277,2025-08-28 23:59:59.999,1.098862e+10,2252826,48243.477,...,-0.483742,-0.478195,49386.800,0.494145,0.505855,-0.011711,0.426923,4877.705523,0.043337,-0.981694
151,2025-08-29,112507.3,112585.7,107411.4,108332.4,179725.172,2025-08-29 23:59:59.999,1.968867e+10,3720115,85606.138,...,1.244865,1.282891,94119.034,0.476317,0.523683,-0.047366,-0.821628,5292.489281,0.048312,-0.259850
152,2025-08-30,108332.4,108879.6,107281.0,108757.9,60656.883,2025-08-30 23:59:59.999,6.572710e+09,1397811,29582.419,...,-1.348869,-1.339517,31074.464,0.487701,0.512299,-0.024598,-0.028936,4702.144808,0.043394,-1.187669


In [4]:
df.to_csv("../Idea_data/future.cal.csv")